In [2]:
# 📦 Install necessary libraries if not already installed (for Colab)
!pip install -q transformers ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.8 MB/s eta 0:00:00


In [4]:


# ⚙️ Enable widgets (for Google Colab)
from google.colab import output
output.enable_custom_widget_manager()

# 🧠 GPT Text Generator (no trimming logic)
from transformers import pipeline, set_seed
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import torch

# Optional: for reproducibility
set_seed(42)

# 🔽 Model Selector Dropdown
model_selector = widgets.Dropdown(
    options=[
        ('Distilled GPT-2', 'distilgpt2'),
        ('GPT-2 Large', 'gpt2-large'),
        ('GPT-Neo 1.3B', 'EleutherAI/gpt-neo-1.3B'),
        ('GPT-Neo 2.7B', 'EleutherAI/gpt-neo-2.7B'),
        ('GPT-J 6B', 'EleutherAI/gpt-j-6B'),
        ('Falcon-7B Instruct (High RAM)', 'tiiuae/falcon-7b-instruct')
    ],
    description='Select Model:',
    value='EleutherAI/gpt-neo-1.3B',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

# 📝 Topic Input
user_input = widgets.Text(
    value='The future of AI is',
    placeholder='Type your topic here...',
    description='Input Topic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# ▶️ Generate Button
generate_button = widgets.Button(
    description="Generate Text",
    button_style='success'
)

# 📤 Output Area
output_area = widgets.Output()

# 🧠 Text Generation Function
def generate_text(_):
    output_area.clear_output()
    prompt = user_input.value.strip()
    model_id = model_selector.value

    if not prompt:
        with output_area:
            display(Markdown("⚠️ **Please enter a topic first.**"))
        return

    with output_area:
        try:
            device = 0 if torch.cuda.is_available() else -1
            print(f"🚀 Loading model `{model_id}` on {'CUDA' if device == 0 else 'CPU'}...")

            generator = pipeline("text-generation", model=model_id, device=device)
            result = generator(
                prompt,
                max_length=250,
                do_sample=True,
                temperature=0.8,
                top_k=50,
                top_p=0.95,
                repetition_penalty=1.5,
                truncation=True,
                pad_token_id=50256
            )[0]["generated_text"]

            display(Markdown(f"### 📝 Generated Text:\n\n{result}"))

            # Copy-to-clipboard button (browser only)
            display(HTML('''
                <button onclick="navigator.clipboard.writeText(document.querySelector('.output_subarea').innerText)">
                    📋 Copy Text
                </button>
            '''))

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                display(Markdown("❌ **CUDA Out of Memory!** Try switching to a smaller model."))
            else:
                display(Markdown(f"⚠️ **Error:** {str(e)}"))

# 🔁 Link button click to function
generate_button.on_click(generate_text)

# 📋 Show UI
display(model_selector, user_input, generate_button, output_area)


Dropdown(description='Select Model:', index=2, layout=Layout(width='60%'), options=(('Distilled GPT-2', 'disti…

Text(value='The future of AI is', description='Input Topic:', layout=Layout(width='80%'), placeholder='Type yo…

Button(button_style='success', description='Generate Text', style=ButtonStyle())

Output()